## Importar Librerias

In [2]:
from psycopg2 import connect, Error
import pandas as pd
import csv
from io import StringIO
import glob
import functools

## Conexion

In [12]:
def getConnection():
    try:
        connection = connect(host='localhost',database='amazon',user='postgres', password='1234', port='5432')   
    except(Exception, Error) as error:
        connection.rollback()
        print("Error: %s" % error)
    return connection

## Insert query

In [13]:
def insert_query(query,data):
    try:
        con = getConnection()
        if con and query != '':
            cursor = con.cursor()
            cursor.execute(query,data)
        con.commit()
    except(Exception, Error) as error:
        print("Error: %s" % error)

## Cargar product_category

In [14]:
files = glob.glob('*.tsv')
query = "insert into category(category_name) values (%s)"

for file in files:
    start = len("amazon_reviews_us_")
    end = file.index("_v1_00.tsv")
    #name = file[start:end].replace("_"," ")  
    name = file[start:end]
    insert_query(query, (name,))

## Cargar a la base de datos

In [15]:
def to_database(dataframe,table):
    try:
        output = StringIO()
        dataframe.to_csv(output, sep='\t', index = False, header = False, quoting=csv.QUOTE_NONE,escapechar='\\')
        output.seek(0)
        connection = getConnection()
        cursor = connection.cursor()
        cursor.copy_from(output,table,null='')
        connection.commit()
        cursor.close()
    except(Exception, Error) as error:
        print("Error: %s" % error)



##  Crear DataFrame Completo

In [6]:
df = pd.concat(map(functools.partial(pd.read_csv, sep='\t',on_bad_lines='skip', quoting = 3), glob.glob('*.tsv')))

## Cargar Customer

In [17]:
df_customers_id = df[["customer_id"]]
df_customers_id = df_customers_id.drop_duplicates(subset="customer_id")

to_database(df_customers_id,"customer")


## Cargar Product

In [18]:
df_products = df[["product_id", "product_parent", "product_title", "product_category"]]
df_products = df_products.drop_duplicates(subset="product_id")

to_database(df_products,"product")

## Cargar Time

In [19]:

df_time = df[["review_date"]]
df_time = df_time.drop_duplicates(subset="review_date").dropna()

to_database(df_time,"time")

## Cargar Review

In [20]:
#Se leen todos los archivos que se encuentren en la carpeta terminados en .tsv
#LOS BOLEANOS EN LA REVIEW HAY QUE CAMBIARLOS

df_review = df[["marketplace", "customer_id", "review_id", "product_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase", "review_headline", "review_body","review_date"]]

to_database(df_review,"review")